In [14]:
from tradealgorithm.tradealgorithm import TradeAlgorithm
from tradealgorithm.tradealgorithmfactory import TradeAlgorithmFactory
from pricer.pricer_factory import PricerFactory
from pricer.pricer import Pricer
from ranker.ranker_factory import RankerFactory
from ranker.ranker import Ranker
from classifier.classifier_factory import ClassifierFactory
from classifier.classifier import Classifier
from datetime import datetime
from tqdm import tqdm
from alpaca_api.alpaca_api import AlpacaApi
from time import sleep
import pandas as pd

In [15]:
trade_algo = TradeAlgorithmFactory.build(TradeAlgorithm.RRR_BETA)
alp = AlpacaApi()

In [16]:
ranker_class = Ranker.NONE
classifier_class = Classifier.NONE
current = False

In [17]:
start = datetime(2001,1,1)
end = datetime(2023,1,1)
current_date = datetime.now()
# week = current_date.week
week = 29
positions = 20

In [18]:
pricer_classes = [] 
# pricer_classes.append(Pricer.DAILY_STOCK_ROLLING)
pricer_classes.append(Pricer.WEEKLY_STOCK_ROLLING)
# pricer_classes.append(Pricer.MONTHLY_STOCK_ROLLING)
# pricer_classes.append(Pricer.QUARTERLY_STOCK_ROLLING)
# pricer_classes.append(Pricer.DAILY_STOCK_WINDOW)
# pricer_classes.append(Pricer.WEEKLY_STOCK_WINDOW)
# pricer_classes.append(Pricer.MONTHLY_STOCK_WINDOW)
# pricer_classes.append(Pricer.QUARTERLY_STOCK_WINDOW)

In [19]:
for pricer_class in tqdm(pricer_classes):
    try:
        trade_algo.initialize(pricer_class,ranker_class,classifier_class,start,end,current_date)
        trade_algo.initialize_classes()
        final = trade_algo.pull_recommendations()
        if final.index.size > 0:
            final = final[final["week"]==week].sort_values("weekly_delta",ascending=False).head(positions)
            trade_algo.db.cloud_connect()
            trade_algo.db.store("proposals",final)
            trade_algo.db.disconnect()
            trade_algo.db.connect()
            trade_algo.db.store("proposals",final)
            trade_algo.db.disconnect()
            account = alp.live_get_account()
            cash = float(account.cash)
            # executing order
            order_data = []
            for row in final.iterrows():
                try:
                    ticker = "BTC/USD" if row[1]["ticker"] == "BTC" else row[1]["ticker"] 
                    amount = round(cash / (positions))
                    print(ticker,amount)
#                     order_data.append(alp.paper_market_order(ticker,amount))
#                     sleep(1)
                except Exception as e:
                    trade_algo.db.cloud_connect()
                    trade_algo.db.store("errors",pd.DataFrame([{"date":str(datetime.now()),"error":str(e)}]))
                    trade_algo.db.disconnect()
            order_dicts = pd.DataFrame([json.loads(order_d.json()) for order_d in order_data])
            trade_algo.db.cloud_connect()
            trade_algo.db.store("orders",order_dicts)
            trade_algo.db.disconnect()
            trade_algo.db.connect()
            trade_algo.db.store("orders",order_dicts)
            trade_algo.db.disconnect()
        trade_algo.db.cloud_connect()
        trade_algo.db.store("iterations",pd.DataFrame([{"date":str(datetime.now()),"status":"complete"}]))
        trade_algo.db.disconnect()
        trade_algo.db.connect()
        trade_algo.db.store("iterations",pd.DataFrame([{"date":str(datetime.now()),"status":"complete"}]))
        trade_algo.db.disconnect()
#         sleep(300)
    except Exception as e:
            trade_algo.db.cloud_connect()
            trade_algo.db.store("iterations",pd.DataFrame([{"date":str(datetime.now()),"status":"incomplete"}]))
            trade_algo.db.store("errors",pd.DataFrame([{"date":str(datetime.now()),"status":"trade","error":str(e)}]))
            trade_algo.db.disconnect()
            trade_algo.db.connect()
            trade_algo.db.store("iterations",pd.DataFrame([{"date":str(datetime.now()),"status":"incomplete"}]))
            trade_algo.db.store("errors",pd.DataFrame([{"date":str(datetime.now()),"status":"trade","error":str(e)}]))
            trade_algo.db.disconnect()

  0%|                                                                                                                                                                               | 0/1 [00:00<?, ?it/s]

CCL 0
NVDA 0
TSLA 0
META 0
NCLH 0
ADBE 0
AVGO 0
RCL 0
MTCH 0
PHM 0
NFLX 0
DAL 0
ISRG 0
LYV 0
DPZ 0
GNRC 0
NOW 0
PAYC 0
NXPI 0
AMZN 0
wsr_n_n_b_r orders documents must be a non-empty list
wsr_n_n_b_r orders documents must be a non-empty list


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.02s/it]


In [ ]:
[]